In [44]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path

import sklearn as skl
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf


---

## Split the Data into Training and Testing Sets

### Step 1: Read the `bank_customer_data.csv` file into a Pandas DataFrame.

In [8]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
customer_data_df = pd.read_csv("/bank_customer_data.csv")

# Review the DataFrame
customer_data_df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [9]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = customer_data_df['Exited']

# Separate the raw X variable data, the features, and drop the labels/outcomes column as well as additional columns not useful in predicting outcomes
X_raw_data = customer_data_df.copy()
X_raw_data.drop(['Exited','Surname','CustomerId','RowNumber'], axis=1, inplace=True)


In [10]:
# Convert/encode columns with categorical feature values into numerical values, save results as final 'X' feature DataFrame
X_encoded = pd.get_dummies(X_raw_data, dtype=int)

X = X_encoded


In [11]:
# Review the y variable Series
print(y[:5])


0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64


In [12]:
# Review the X variable DataFrame
X.head(10)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,1,0
5,645,44,8,113755.78,2,1,0,149756.71,0,0,1,0,1
6,822,50,7,0.00,2,1,1,10062.80,1,0,0,0,1
7,376,29,4,115046.74,4,1,0,119346.88,0,1,0,1,0
8,501,44,4,142051.07,2,0,1,74940.50,1,0,0,0,1
9,684,27,2,134603.88,1,1,1,71725.73,1,0,0,0,1


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [13]:
# Check the balance of our target values
y.value_counts()


0    7963
1    2037
Name: Exited, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [14]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## 1: ORIGINAL DATA Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [15]:
# Define training and testing data sets for this first version of the model and label with '1'

X_train1 = X_train
X_test1 = X_test
y_train1 = y_train
y_test1 = y_test


In [16]:
# Instantiate the Logistic Regression model with a random_state parameter of 1, labeled with '1' for this first version of the model
classifier1 = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier1.fit(X_train1, y_train1)


LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [17]:
# Make a prediction using the testing data
predictions1 = classifier1.predict(X_test1)
pd.DataFrame({"Prediction": predictions1, "Actual": y_test1}).head(10)


,Prediction,Actual
9953,0,0
3850,0,0
4962,0,0
3886,0,0
5437,0,0
8517,0,0
2041,0,0
1989,0,0
1933,1,0
9984,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [18]:
# Print the balanced_accuracy score of the 1st model
print(balanced_accuracy_score(y_test1, predictions1))


0.5135295260295261


In [19]:
# Generate a confusion matrix for the 1st model
cm1 = confusion_matrix(y_test1, predictions1)
cm1_df = pd.DataFrame(
    cm1, index=['actual_retained', 'actual_churned'], columns=['predicted_retained', 'predicted_churned']
)

print(cm1_df)


                 predicted_retained  predicted_churned
actual_retained                1946                 34
actual_churned                  497                 23


In [20]:
# Print the classification report for the 1st model
target_names = ['retained_customer', 'churned_customer']

cr1 = classification_report(y_test1, predictions1,
                            target_names=target_names)

print(cr1)


                   precision    recall  f1-score   support

retained_customer       0.80      0.98      0.88      1980
 churned_customer       0.40      0.04      0.08       520

         accuracy                           0.79      2500
        macro avg       0.60      0.51      0.48      2500
     weighted avg       0.71      0.79      0.71      2500



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model is quite effective at predicting both healthy and high-risk loans, as indicated by the strong f1-scores for each (as well as the strong underlying precision and recall scores). However, in this case the logistic regression model appears to be significantly less effective at predicting high-risk loans than at predicting healthy loans, based on the respective f1-scores of 0.88 vs 1.00). This may be partially due to the dataset being quite unbalanced, with a significant majority of loans being healthy, but for business purposes it may be ideal to find a model that skews in the other direction - 'over-flagging' loans for further review may be better than 'under-flagging', depending on the cost vs. benefit of manually reviewing potentially high-risk loans.

---

## 2: RESAMPLED/OVERSAMPLED DATA Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [21]:
# Instantiate the random oversampler model with a random_state parameter of 1
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_R, y_R = ros.fit_resample(X_train, y_train)


In [22]:
# Count the distinct values of the resampled labels data
y_R.value_counts()


0    5983
1    5983
Name: Exited, dtype: int64

In [23]:
# Define training and testing data sets for this second version of the model and label with '2'

X_train2 = X_R
X_test2 = X_test
y_train2 = y_R
y_test2 = y_test


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [24]:
# Instantiate the Logistic Regression model with a random_state parameter of 1, labeled with '2' for this first version of the model
classifier2 = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier2.fit(X_train2, y_train2)

# Make a prediction using the testing data
predictions2 = classifier2.predict(X_test2)
pd.DataFrame({"Prediction": predictions2, "Actual": y_test2}).head(10)


,Prediction,Actual
9953,1,0
3850,0,0
4962,1,0
3886,0,0
5437,0,0
8517,0,0
2041,0,0
1989,0,0
1933,1,0
9984,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [25]:
# Print the balanced_accuracy score of the 2nd model
print(balanced_accuracy_score(y_test, predictions2))


0.673931623931624


In [26]:
# Generate a confusion matrix for the 2nd model
cm2 = confusion_matrix(y_test2, predictions2)
cm2_df = pd.DataFrame(
    cm2, index=['actual_healthy', 'actual_at_risk'], columns=['predicted_healthy', 'predicted_at_risk']
)

print(cm2_df)


                predicted_healthy  predicted_at_risk
actual_healthy               1298                682
actual_at_risk                160                360


In [27]:
# Print the classification report for the 2nd model

cr2 = classification_report(y_test2, predictions2,
                            target_names=target_names)

print(cr2)


                   precision    recall  f1-score   support

retained_customer       0.89      0.66      0.76      1980
 churned_customer       0.35      0.69      0.46       520

         accuracy                           0.66      2500
        macro avg       0.62      0.67      0.61      2500
     weighted avg       0.78      0.66      0.69      2500



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** While the original logistic regression model is quite effective at predicting both healthy and high-risk loans, refitting a logistic regression model to the re-/over-sampled data produces a solidly more reliable model/prediction with a higher f1-score for predicting high-risk loans.

From a business perspective this over-sampling-based model may be much more ideal, as the  model 'missed' 52 fewer high-risk loans (just 1/14 (or ~7%) as many high-risk loans as the original model 'missed'), while only mistakenly predicting 14 additional healthy loans as being high-risk compared to the original model. This new model seems likely to be much more favorable from a profitabilty perspective and I would likely recommend that the bank put it into use for flagging potentially high-risk loans for manual/human review (though again it depends on the cost of manual review vs the benefit of identifying high-risk loans early).

## SCALED DATA Predict a Logistic Regression Model with Scaled Training Data

In [28]:
# Instantiate the StandardScaler model to normalize the original training and test data
sc = StandardScaler()


In [29]:
# Define training and testing data sets for this third version of the model, labled with '3', using the StandardScaler model fit to X_train to transform both X_train and X_test

X_train3 = sc.fit_transform(X_train)
X_test3 = sc.transform(X_test)
y_train3 = y_train
y_test3 = y_test


In [30]:
# Instantiate the Logistic Regression model with a random_state parameter of 1, labeled with '3' for this third version of the model
classifier3 = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier3.fit(X_train3, y_train3)


LogisticRegression(random_state=1)

In [31]:
# Make a prediction using the testing data
predictions3 = classifier3.predict(X_test3)
pd.DataFrame({"Prediction": predictions3, "Actual": y_test3}).head(10)


,Prediction,Actual
9953,0,0
3850,0,0
4962,0,0
3886,0,0
5437,0,0
8517,0,0
2041,0,0
1989,0,0
1933,1,0
9984,0,0


In [32]:
# Print the balanced_accuracy score of the 3rd model
print(balanced_accuracy_score(y_test3, predictions3))


0.593405205905206


In [33]:
# Generate a confusion matrix for the 3rd model
cm3 = confusion_matrix(y_test3, predictions3)
cm3_df = pd.DataFrame(
    cm3, index=['actual_retained', 'actual_churned'], columns=['predicted_retained', 'predicted_churned']
)

print(cm3_df)


                 predicted_retained  predicted_churned
actual_retained                1912                 68
actual_churned                  405                115


In [34]:
# Print the classification report for the 3rd model
target_names = ['retained_customer', 'churned_customer']

cr3 = classification_report(y_test3, predictions3,
                            target_names=target_names)

print(cr3)


                   precision    recall  f1-score   support

retained_customer       0.83      0.97      0.89      1980
 churned_customer       0.63      0.22      0.33       520

         accuracy                           0.81      2500
        macro avg       0.73      0.59      0.61      2500
     weighted avg       0.78      0.81      0.77      2500



## SCALED+RESAMPLED/OVERSAMPLED DATA Predict a Logistic Regression Model with Scaled+Oversampled Training Data

In [35]:
# Instantiate the random oversampler model with a random_state parameter of 1
ros = RandomOverSampler(random_state=1)

# Fit the scaled training data from the prior version of the model to the random_oversampler model
X_R, y_R = ros.fit_resample(X_train3, y_train3)


In [36]:
# Count the distinct values of the resampled labels/outcomes data
y_R.value_counts()


0    5983
1    5983
Name: Exited, dtype: int64

In [37]:
# Define training and testing data sets for this fourth version of the model and label with '4'

X_train4 = X_R
X_test4 = X_test3
y_train4 = y_R
y_test4 = y_test


In [38]:
# Instantiate the Logistic Regression model with a random_state parameter of 1
classifier4 = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier4.fit(X_train4, y_train4)


LogisticRegression(random_state=1)

In [39]:
# Make a prediction using the testing data
predictions4 = classifier4.predict(X_test4)
pd.DataFrame({"Prediction": predictions4, "Actual": y_test4}).head(10)


,Prediction,Actual
9953,0,0
3850,0,0
4962,0,0
3886,0,0
5437,0,0
8517,0,0
2041,1,0
1989,0,0
1933,1,0
9984,0,0


In [40]:
# Print the balanced_accuracy score of the 4th model
print(balanced_accuracy_score(y_test4, predictions4))


0.7118589743589743


In [41]:
# Generate a confusion matrix for the 4th model
cm4 = confusion_matrix(y_test4, predictions4)
cm4_df = pd.DataFrame(
    cm4, index=['actual_retained', 'actual_churned'], columns=['predicted_retained', 'predicted_churned']
)

print(cm4_df)


                 predicted_retained  predicted_churned
actual_retained                1452                528
actual_churned                  161                359


In [42]:
# Print the classification report for the 4th model
target_names = ['retained_customer', 'churned_customer']

cr4 = classification_report(y_test4, predictions4,
                            target_names=target_names)

print(cr4)


                   precision    recall  f1-score   support

retained_customer       0.90      0.73      0.81      1980
 churned_customer       0.40      0.69      0.51       520

         accuracy                           0.72      2500
        macro avg       0.65      0.71      0.66      2500
     weighted avg       0.80      0.72      0.75      2500



## NN MODEL: Build and Train a Neural Network Model (using Scaled Training Data)

In [51]:
# Create the Keras Sequential model
nn_model1 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model1.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=13))

# Add the output layer that uses a probability activation function
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1)                 14        
                                                                 
 dense_7 (Dense)             (None, 1)                 2         
                                                                 
Total params: 16 (64.00 Byte)
Trainable params: 16 (64.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
# Compile the Sequential model together and customize metrics
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [119]:
# Fit the model to the training data
fit_nn_model1 = nn_model1.fit(X_train3, y_train3, epochs=42)

Epoch 1/42
235/235 [==============================] - 1s 4ms/step - loss: 0.4313 - accuracy: 0.8128
Epoch 2/42
235/235 [==============================] - 1s 4ms/step - loss: 0.4313 - accuracy: 0.8128
Epoch 3/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4312 - accuracy: 0.8127
Epoch 4/42
235/235 [==============================] - 1s 5ms/step - loss: 0.4310 - accuracy: 0.8117
Epoch 5/42
235/235 [==============================] - 2s 9ms/step - loss: 0.4310 - accuracy: 0.8137
Epoch 6/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4308 - accuracy: 0.8124
Epoch 7/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4308 - accuracy: 0.8117
Epoch 8/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4307 - accuracy: 0.8131
Epoch 9/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4305 - accuracy: 0.8129
Epoch 10/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4304 - accuracy: 0.8128

## NN MODEL + Oversampling: Build and Train a Neural Network Model (using Scaled+Oversampled Training Data)

In [55]:
# Create the Keras Sequential model
nn_model2 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=13))

# Add the output layer that uses a probability activation function
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 1)                 14        
                                                                 
 dense_9 (Dense)             (None, 1)                 2         
                                                                 
Total params: 16 (64.00 Byte)
Trainable params: 16 (64.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
# Compile the Sequential model together and customize metrics
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [58]:
# Fit the model to the training data
fit_nn_model2 = nn_model2.fit(X_train4, y_train4, epochs=47)

Epoch 1/47
374/374 [==============================] - 1s 3ms/step - loss: 0.5799 - accuracy: 0.7098
Epoch 2/47
374/374 [==============================] - 1s 2ms/step - loss: 0.5797 - accuracy: 0.7103
Epoch 3/47
374/374 [==============================] - 1s 2ms/step - loss: 0.5795 - accuracy: 0.7103
Epoch 4/47
374/374 [==============================] - 1s 2ms/step - loss: 0.5794 - accuracy: 0.7101
Epoch 5/47
374/374 [==============================] - 1s 2ms/step - loss: 0.5794 - accuracy: 0.7096
Epoch 6/47
374/374 [==============================] - 1s 3ms/step - loss: 0.5792 - accuracy: 0.7099
Epoch 7/47
374/374 [==============================] - 1s 2ms/step - loss: 0.5791 - accuracy: 0.7098
Epoch 8/47
374/374 [==============================] - 1s 2ms/step - loss: 0.5791 - accuracy: 0.7102
Epoch 9/47
374/374 [==============================] - 1s 2ms/step - loss: 0.5790 - accuracy: 0.7113
Epoch 10/47
374/374 [==============================] - 1s 2ms/step - loss: 0.5789 - accuracy: 0.7100

## NN MODEL + 1 Hidden Layer: Build and Train a Neural Network Model (using Scaled+Oversampled Training Data)

In [69]:
# Create the Keras Sequential model
nn_model3 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model3.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=13))

# Add a second Dense layer, to be a hidden layer
nn_model3.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 1)                 14        
                                                                 
 dense_19 (Dense)            (None, 4)                 8         
                                                                 
 dense_20 (Dense)            (None, 1)                 5         
                                                                 
Total params: 27 (108.00 Byte)
Trainable params: 27 (108.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
# Compile the Sequential model together and customize metrics
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [71]:
# Fit the model to the training data
fit_nn_model3 = nn_model3.fit(X_train3, y_train3, epochs=42)

Epoch 1/42
235/235 [==============================] - 1s 2ms/step - loss: 0.5949 - accuracy: 0.7769
Epoch 2/42
235/235 [==============================] - 1s 2ms/step - loss: 0.4942 - accuracy: 0.7977
Epoch 3/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4727 - accuracy: 0.7977
Epoch 4/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4628 - accuracy: 0.7977
Epoch 5/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4557 - accuracy: 0.7977
Epoch 6/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4494 - accuracy: 0.7977
Epoch 7/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4444 - accuracy: 0.7977
Epoch 8/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4403 - accuracy: 0.7977
Epoch 9/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4369 - accuracy: 0.7977
Epoch 10/42
235/235 [==============================] - 0s 2ms/step - loss: 0.4342 - accuracy: 0.7977

## NN MODEL + More Layers + More Units: Build and Train a Neural Network Model (using Scaled+Oversampled Training Data)

In [87]:
# Create the Keras Sequential model
nn_model4 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model4.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=13))

# Add a second Dense layer, to be a hidden layer
nn_model4.add(tf.keras.layers.Dense(units=10, activation="linear"))

# Add a third Dense layer, to be another hidden layer
nn_model4.add(tf.keras.layers.Dense(units=6, activation="tanh"))

# Add the output layer that uses a probability activation function
nn_model4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model4.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 1)                 14        
                                                                 
 dense_42 (Dense)            (None, 10)                20        
                                                                 
 dense_43 (Dense)            (None, 6)                 66        
                                                                 
 dense_44 (Dense)            (None, 1)                 7         
                                                                 
Total params: 107 (428.00 Byte)
Trainable params: 107 (428.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [88]:
# Compile the Sequential model together and customize metrics
nn_model4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [89]:
# Fit the model to the training data
fit_nn_model4 = nn_model4.fit(X_train3, y_train3, epochs=42)

Epoch 1/42
235/235 [==============================] - 2s 3ms/step - loss: 0.5178 - accuracy: 0.7977
Epoch 2/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4947 - accuracy: 0.7977
Epoch 3/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4911 - accuracy: 0.7977
Epoch 4/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4861 - accuracy: 0.7977
Epoch 5/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4813 - accuracy: 0.7977
Epoch 6/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4762 - accuracy: 0.7977
Epoch 7/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4705 - accuracy: 0.7977
Epoch 8/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4645 - accuracy: 0.7977
Epoch 9/42
235/235 [==============================] - 1s 3ms/step - loss: 0.4593 - accuracy: 0.7977
Epoch 10/42
235/235 [==============================] - 1s 4ms/step - loss: 0.4551 - accuracy: 0.7977

## NN MODEL same as above but with oversampling too... + More Layers + More Units: Build and Train a Neural Network Model (using Scaled+Oversampled Training Data)

In [92]:
# Create the Keras Sequential model
nn_model5 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model5.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=13))

# Add a second Dense layer, to be a hidden layer
nn_model5.add(tf.keras.layers.Dense(units=10, activation="linear"))

# Add a third Dense layer, to be another hidden layer
nn_model5.add(tf.keras.layers.Dense(units=6, activation="tanh"))

# Add the output layer that uses a probability activation function
nn_model5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model5.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_49 (Dense)            (None, 1)                 14        
                                                                 
 dense_50 (Dense)            (None, 10)                20        
                                                                 
 dense_51 (Dense)            (None, 6)                 66        
                                                                 
 dense_52 (Dense)            (None, 1)                 7         
                                                                 
Total params: 107 (428.00 Byte)
Trainable params: 107 (428.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [93]:
# Compile the Sequential model together and customize metrics
nn_model5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [94]:
# Fit the model to the training data
fit_nn_model5 = nn_model5.fit(X_train4, y_train4, epochs=42)

Epoch 1/42
374/374 [==============================] - 2s 2ms/step - loss: 0.6892 - accuracy: 0.5197
Epoch 2/42
374/374 [==============================] - 1s 3ms/step - loss: 0.6526 - accuracy: 0.6286
Epoch 3/42
374/374 [==============================] - 1s 3ms/step - loss: 0.6172 - accuracy: 0.6689
Epoch 4/42
374/374 [==============================] - 1s 3ms/step - loss: 0.5996 - accuracy: 0.6873
Epoch 5/42
374/374 [==============================] - 1s 3ms/step - loss: 0.5922 - accuracy: 0.6941
Epoch 6/42
374/374 [==============================] - 1s 2ms/step - loss: 0.5875 - accuracy: 0.6998
Epoch 7/42
374/374 [==============================] - 1s 2ms/step - loss: 0.5846 - accuracy: 0.7034
Epoch 8/42
374/374 [==============================] - 1s 2ms/step - loss: 0.5827 - accuracy: 0.7058
Epoch 9/42
374/374 [==============================] - 1s 2ms/step - loss: 0.5816 - accuracy: 0.7096
Epoch 10/42
374/374 [==============================] - 1s 2ms/step - loss: 0.5812 - accuracy: 0.7119

## NN MODEL 6: more layers, more units... + More Layers + More Units: Build and Train a Neural Network Model (using Scaled+Oversampled Training Data)

In [130]:
# Create the Keras Sequential model
nn_model6 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model6.add(tf.keras.layers.Dense(units=1, activation="sigmoid", input_dim=13))

# Add a second Dense layer, to be a hidden layer
nn_model6.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Add a third Dense layer, to be a hidden layer
nn_model6.add(tf.keras.layers.Dense(units=40, activation="sigmoid"))

# Add a fourth Dense layer, to be another hidden layer
nn_model6.add(tf.keras.layers.Dense(units=20, activation="linear"))

# Add the output layer that uses a probability activation function
nn_model6.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model6.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_120 (Dense)           (None, 1)                 14        
                                                                 
 dense_121 (Dense)           (None, 40)                80        
                                                                 
 dense_122 (Dense)           (None, 40)                1640      
                                                                 
 dense_123 (Dense)           (None, 20)                820       
                                                                 
 dense_124 (Dense)           (None, 1)                 21        
                                                                 
Total params: 2575 (10.06 KB)
Trainable params: 2575 (10.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [131]:
# Compile the Sequential model together and customize metrics
nn_model6.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [132]:
# Fit the model to the training data
fit_nn_model6 = nn_model6.fit(X_train3, y_train3, epochs=17)

Epoch 1/17
235/235 [==============================] - 3s 2ms/step - loss: 0.5583 - accuracy: 0.7580
Epoch 2/17
235/235 [==============================] - 1s 2ms/step - loss: 0.5029 - accuracy: 0.7977
Epoch 3/17
235/235 [==============================] - 1s 2ms/step - loss: 0.4834 - accuracy: 0.7977
Epoch 4/17
235/235 [==============================] - 1s 2ms/step - loss: 0.4441 - accuracy: 0.8035
Epoch 5/17
235/235 [==============================] - 1s 3ms/step - loss: 0.4338 - accuracy: 0.8073
Epoch 6/17
235/235 [==============================] - 1s 2ms/step - loss: 0.4310 - accuracy: 0.8056
Epoch 7/17
235/235 [==============================] - 1s 4ms/step - loss: 0.4312 - accuracy: 0.8071
Epoch 8/17
235/235 [==============================] - 1s 4ms/step - loss: 0.4292 - accuracy: 0.8076
Epoch 9/17
235/235 [==============================] - 1s 4ms/step - loss: 0.4295 - accuracy: 0.8048
Epoch 10/17
235/235 [==============================] - 1s 4ms/step - loss: 0.4300 - accuracy: 0.8033

## NN MODEL 7: fewer layers...: Build and Train a Neural Network Model (using Scaled+Oversampled Training Data)

In [155]:
# Create the Keras Sequential model
nn_model7 = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model7.add(tf.keras.layers.Dense(units=1, activation="tanh", input_dim=13))

# Add a second Dense layer, to be a hidden layer
nn_model7.add(tf.keras.layers.Dense(units=13, activation="relu"))

# Add a second Dense layer, to be a hidden layer
nn_model7.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model7.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model7.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_141 (Dense)           (None, 1)                 14        
                                                                 
 dense_142 (Dense)           (None, 13)                26        
                                                                 
 dense_143 (Dense)           (None, 5)                 70        
                                                                 
 dense_144 (Dense)           (None, 1)                 6         
                                                                 
Total params: 116 (464.00 Byte)
Trainable params: 116 (464.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [156]:
# Compile the Sequential model together and customize metrics
nn_model7.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [157]:
# Fit the model to the training data
fit_nn_model7 = nn_model7.fit(X_train3, y_train3, epochs=17)

Epoch 1/17
235/235 [==============================] - 2s 2ms/step - loss: 0.5616 - accuracy: 0.7936
Epoch 2/17
235/235 [==============================] - 1s 3ms/step - loss: 0.5034 - accuracy: 0.7977
Epoch 3/17
235/235 [==============================] - 1s 2ms/step - loss: 0.4939 - accuracy: 0.7977
Epoch 4/17
235/235 [==============================] - 1s 2ms/step - loss: 0.4616 - accuracy: 0.7977
Epoch 5/17
235/235 [==============================] - 1s 3ms/step - loss: 0.4350 - accuracy: 0.7976
Epoch 6/17
235/235 [==============================] - 1s 3ms/step - loss: 0.4297 - accuracy: 0.7971
Epoch 7/17
235/235 [==============================] - 1s 4ms/step - loss: 0.4278 - accuracy: 0.7951
Epoch 8/17
235/235 [==============================] - 1s 4ms/step - loss: 0.4278 - accuracy: 0.7980
Epoch 9/17
235/235 [==============================] - 1s 5ms/step - loss: 0.4276 - accuracy: 0.7953
Epoch 10/17
235/235 [==============================] - 1s 5ms/step - loss: 0.4276 - accuracy: 0.7947

In [161]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model7.evaluate(X_test3,y_test3,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 1s - loss: 0.4238 - accuracy: 0.8044 - 991ms/epoch - 13ms/step
Loss: 0.42378199100494385, Accuracy: 0.8044000267982483


In [162]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model6.evaluate(X_test3,y_test3,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 0s - loss: 0.4468 - accuracy: 0.7920 - 272ms/epoch - 3ms/step
Loss: 0.44683241844177246, Accuracy: 0.7919999957084656


In [164]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model5.evaluate(X_test4,y_test4,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 0s - loss: 0.5740 - accuracy: 0.7048 - 420ms/epoch - 5ms/step
Loss: 0.574043333530426, Accuracy: 0.704800009727478


In [165]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model4.evaluate(X_test3,y_test3,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 0s - loss: 0.6358 - accuracy: 0.7920 - 347ms/epoch - 4ms/step
Loss: 0.6358230113983154, Accuracy: 0.7919999957084656


In [166]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test3,y_test3,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 1s - loss: 0.4240 - accuracy: 0.7920 - 529ms/epoch - 7ms/step
Loss: 0.423982709646225, Accuracy: 0.7919999957084656


In [167]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test4,y_test4,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 0s - loss: 0.5615 - accuracy: 0.7192 - 314ms/epoch - 4ms/step
Loss: 0.5615013837814331, Accuracy: 0.7192000150680542


In [169]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test3,y_test3,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 0s - loss: 0.4293 - accuracy: 0.8116 - 227ms/epoch - 3ms/step
Loss: 0.42926910519599915, Accuracy: 0.8116000294685364
